In [1]:
import pandas as pd
import numpy as np
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="72eb7803-e874-44cb-b6d9-33f2fa3eb88c",
    resource_group_name="vpoutotrg",
    workspace_name="mlstudio-groupe4"
)

data_asset = ml_client.data.get("allo_cine", version="11")

df = pd.read_parquet(data_asset.path)
df.head()

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,actors,critics_score,date,directors,editor,french_boxoffice,french_first_week_boxoffice,french_visa,genre,langage,length,nationality,title,url,us_boxoffice,us_first_week_boxoffice,viewers_score,vo_title
0,Arieh Worthalter|Arthur Harari|Stéphan Guérin-...,4.4,2023-09-27,Cédric Kahn,Ad Vitam,351881,110011.0,157303,Policier|Drame|Historique|Judiciaire,Français,116,France,Le Procès Goldman,/article/fichearticle_gen_carticle=1000093547....,NaN,NaN,3.9,None
1,Monica Bellucci|Vincent Cassel|Albert Dupontel...,3.5,2020-08-26,Gaspar Noé,Carlotta Films,5632,2905.0,153336,Drame|Thriller,Anglais|Français|Italien|Espagnol,90,France,Irréversible - Inversion Intégrale,/article/fichearticle_gen_carticle=18692477.html,NaN,NaN,--,None
2,Thom Hoffman,2.8,2024-08-07,Richard Claus|Karsten Kiilerich,Le Pacte,130734,39119.0,160622,Aventure|Animation|Comédie|Famille,Néerlandais,84,Danemark|France|Allemagne|Pays-Bas,Petit Panda en Afrique,/article/fichearticle_gen_carticle=1000096064....,NaN,NaN,3.0,Panda Bear in Africa
3,Lou de Laâge|Raphaël Personnaz|Isabelle Carré|...,3.3,2022-12-21,Olivier Treiner,SND,200631,64039.0,152607,Drame,Français,120,France,Le Tourbillon de la vie,/article/fichearticle_gen_carticle=1000006465....,NaN,NaN,3.8,None
4,Michael B. Jordan|Jamie Foxx|Brie Larson|Rob M...,3.0,2020-01-29,Destin Daniel Cretton,Warner Bros. France,315693,113153.0,152118,Biopic|Drame,Anglais,137,U.S.A.,La Voie de la justice,/video/player_gen_cmedia=19586793&cfilm=239735...,27054597.0,9713228.0,4.1,Just Mercy


In [2]:
df.shape

(9158, 18)

In [3]:
df.dtypes

actors                          object
critics_score                   object
date                            object
directors                       object
editor                          object
french_boxoffice                 int64
french_first_week_boxoffice    float64
french_visa                     object
genre                           object
langage                         object
length                          object
nationality                     object
title                           object
url                             object
us_boxoffice                   float64
us_first_week_boxoffice        float64
viewers_score                   object
vo_title                        object
dtype: object

In [4]:
df['viewers_score']= pd.to_numeric(df['viewers_score'], errors='coerce')
df['critics_score']= pd.to_numeric(df['critics_score'], errors='coerce')
df['french_boxoffice']= pd.to_numeric(df['french_boxoffice'], errors='coerce')
df['us_first_week_boxoffice']= pd.to_numeric(df['us_first_week_boxoffice'], errors='coerce')
df = df[df['french_boxoffice'].notna()]

In [5]:
df['length']= pd.to_numeric(df['length'], errors='coerce')
df['length'] = df['length'].apply(lambda x : np.nan if x == 0 else x )

In [6]:
df['date']= pd.to_datetime(df['date'], errors='coerce')
df = df[df['date'].notna()]

In [7]:
df['genre'] = df['genre'].str.split('|')
df['actors'] = df['actors'].str.split('|')
df['nationality'] = df['nationality'].str.split('|')
df['langage'] = df['langage'].str.split('|') 	
df['directors'] = df['directors'].str.split('|') 	

In [8]:
feature_of_interest = [
    'actors',
    'critics_score',
    'date',
    'directors',
    'editor',
    'genre',
    'langage',
    'length',
    'nationality',
    'viewers_score',
    'us_first_week_boxoffice'
]

info_film = ['french_visa', 'title', 'vo_title', 'url']


target = 'french_first_week_boxoffice'
data = feature_of_interest
data.append(target)

data.extend(info_film)
df_to_train = df[data]

In [9]:
df_to_train

,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,NaN,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,NaN,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,NaN,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,NaN,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9153,"[Brad Pitt, Joey King, Aaron Taylor-Johnson, B...",3.2,2022-08-03,[David Leitch],Sony Pictures Releasing France,"[Action, Thriller]",[Anglais],127.0,[U.S.A.],3.9,30125000.0,491067.0,157053,Bullet Train,None,/video/player_gen_cmedia=19596933&cfilm=284846...
9154,"[Nicholas Hoult, J.K. Simmons, Kiefer Sutherla...",3.8,2024-10-30,[Clint Eastwood],Warner Bros. France,"[Drame, Thriller]",[Anglais],114.0,[U.S.A.],3.9,NaN,453813.0,-,Juré n°2,Juror #2,/video/player_gen_cmedia=20609934&cfilm=314868...
9155,"[Jacques Gamblin, Pascale Arbillot, Pablo Paul...",3.5,2022-05-11,[François Uzan],UGC Distribution,"[Comédie, Romance]",[Français],96.0,[France],2.8,NaN,37740.0,153600,On sourit pour la photo,None,/article/fichearticle_gen_carticle=18709697.html
9156,"[Alberto Ammann, Bruna Cusí, Ben Temple, Laura...",3.6,2024-05-01,"[Juan Sebastián Vásquez, Alejandro Rojas]",Condor Distribution,"[Drame, Thriller]",[Espagnol],77.0,[Espagne],3.8,NaN,44476.0,161784,Border Line,Upon Entry (La llegada),/video/player_gen_cmedia=20601439&cfilm=309465...


In [10]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import pyarrow as pa
import pyarrow.parquet as pq

df_to_blob = df_to_train
path = 'allocine_spider_clean.parquet'
file_name= "allo_cine" # Le même nom qu'à la step d'avant
version = "2.2.1" # Ici je me dit qu'on pourait se mettre d'accord sur lesquels sont les notres, genres 2.truc c'est vic, 3.truc ludi, 4.truc raouf ?
description = "Correctif petitbug, Pour entrainer les modèles, liste complète des acteurices, ajout du box office américain"

table = pa.Table.from_pandas(df_to_blob)
pq.write_table(table, path)
new_data_asset = Data(
    path=path,
    type=AssetTypes.URI_FILE,
    name=file_name, 
    version=version,           
    description=description
)

ml_client.data.create_or_update(new_data_asset)

HttpResponseError: (UserError) A data version with this name and version already exists. If you are trying to create a new data version, use a different name or version. If you are trying to update an existing data version, the existing asset's data uri cannot be changed. Only tags, description, and isArchived can be updated.
Code: UserError
Message: A data version with this name and version already exists. If you are trying to create a new data version, use a different name or version. If you are trying to update an existing data version, the existing asset's data uri cannot be changed. Only tags, description, and isArchived can be updated.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "0246ee7525479ebe6e2bdc6cdd31edd6",
        "request": "8acdc80028229f1e"
    }
}Type: Environment
Info: {
    "value": "francecentral"
}Type: Location
Info: {
    "value": "francecentral"
}Type: Time
I